# Create the features DF
* using by_postal_code dataset

# Import

In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import math 
from sklearn.model_selection import train_test_split

from scipy import sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [2]:
RANDOM_STATE = 24

In [3]:
res_df = pd.read_pickle ('/home/adam/Steph_C/my_thesis/data/ORI_by_postal_code_s_dropped.pkl').reset_index(drop=True)
venue_df = pd.read_pickle ('../Data/yelp/other_venues.pkl').reset_index(drop=True)
all_df = pd.read_pickle('../Data/yelp/restaurant_only.pkl')

In [4]:
print(res_df.shape , venue_df.shape , all_df.shape)

(192547, 20) (700010, 20) (2112553, 20)


In [5]:
# drop duplicates
df = pd.concat([all_df,venue_df]) # for check in calculation
res_df = res_df.drop_duplicates(subset=['name','postal_code']).reset_index(drop=True)
venue_df = venue_df.drop_duplicates(subset=['name','postal_code']).reset_index(drop=True)
all_df = all_df.drop_duplicates(subset=['name','postal_code']).reset_index(drop=True)

In [6]:
# drop postal_codes not in U.S
drop_postal = [i for i in range(len(all_df)) if not all_df.postal_code[i].isdigit()]
all_df = all_df.drop(drop_postal).reset_index(drop=True)

In [7]:
print(res_df.shape , venue_df.shape , df.shape , all_df.shape)

(1848, 20) (14791, 20) (2812563, 20) (19901, 20)


In [8]:
len(Counter(res_df.name))

405

In [9]:
# create relevance score
cnt = 0
new_df = pd.DataFrame()
for i in Counter(res_df.name):
    tmp = res_df[res_df.name==i].reset_index(drop=True)
    tmp['relevance']=''
    score = 12
    for j in range(len(tmp)):
        tmp['relevance'][j]=score
        score -=1
    new_df = pd.concat([new_df,tmp])

# check the shape
if new_df.shape[0] != res_df.shape[0]:
    print(f'There is a mistake creating the relevance score')
else:
    print(f'Relevance score added')
res_df = new_df

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Relevance score added


In [10]:
# create a overall postal_code features
# postal code and feature dict
postal_code_feature_dict = {}

for postal in Counter(all_df.postal_code):
    
    postal_code_feature_dict[postal]={}
    tmp = df[df.postal_code == postal].reset_index(drop=True)
    tmp_venue = venue_df[venue_df == postal].reset_index(drop=True)
    
    # density 
    postal_code_feature_dict[postal]['density'] = len(Counter(tmp.name))
    
    # neighborhood_entropy
    entropy_sum = 0
    for category in Counter(tmp.categories):
        entropy_sum+=(len(Counter(tmp[tmp.categories==category].name))/len(Counter(tmp.name)))\
        *np.log(len(Counter(tmp[tmp.categories==category].name))/len(Counter(tmp.name)))
    postal_code_feature_dict[postal]['entropy'] = -entropy_sum
    
    # area popularity
    postal_code_feature_dict[postal]['area_pop'] = len(tmp)
    
    # traffic accessibility + complementary
    transportation_cnt = 0
    dep_cnt = 0
    parking_cnt =0
    
    store_cnt = len(Counter(tmp_venue.name))
    
    for i in range(len(tmp_venue)):
        try:
            if 'Public Transportation' in tmp_venue.categories[i]:
                transportation_cnt+=1
            if 'Department Stores' in tmp_venue.categories[i]:
                dep_cnt+=1
            if 'Parking' in tmp_venue.categories[i]:
                parking_cnt+=1
        except:
            pass
    
    postal_code_feature_dict[postal]['accessibility'] = transportation_cnt
    if store_cnt >1:
        postal_code_feature_dict[postal]['complementary'] = (2*(dep_cnt+parking_cnt) )/(store_cnt*(store_cnt-1)) 
    else:
        postal_code_feature_dict[postal]['complementary']=0

In [15]:
len(postal_code_feature_dict)

750

In [ ]:
# # # to check the features
# postal_df = pd.DataFrame(postal_code_feature_dict).T.reset_index().rename(columns={'index': 'postal_code'})
# postal_df
# # Counter(postal_df.complementary)

# Train Test Split with Feature Engineering

In [12]:
def create_new (new ,category,postal, all_df, postal_df):
    
    new = new.drop(columns=['density', 'entropy','area_pop', 'accessibility', 'complementary'])
    new['postal_code'] = postal
    new = pd.DataFrame(new.merge(postal_df, on='postal_code', how='left'))
    new['relevance'] = 0
    # add competitiveness
    new['competitiveness']=''
    place = all_df[all_df.postal_code == postal].reset_index(drop=True)
    new['competitiveness'] = -(len(Counter(place[place.categories == category].name))\
                                  /len(Counter(place.name)))
    # TO-DO Add reviews
    
    return new

## Pointwise

In [30]:
# # POINTWISE v3
# 2-4 間分店用方法一取負樣本，其他參照方法二（解決2-4間分店開在非常遙遠的地方）
# # split train test 

train_df = pd.DataFrame()
test_df = pd.DataFrame()

for i in Counter(res_df.name):
    
    max_postal = max(tmp.postal_code.astype(int))
    category = tmp.categories[0]
    neg_df = pd.DataFrame()
    
    tmp = res_df[res_df.name==i].sort_values(['postal_code'],ascending = [False]).reset_index(drop=True)
    postal_df = pd.DataFrame(postal_code_feature_dict).T.reset_index().rename(columns={'index': 'postal_code'})
    postal_df = postal_df.sort_values(['postal_code'],ascending = [True]).reset_index(drop=True)
    postal_codes_list = list(postal_df.postal_code)
    
    # remove the known postal codes
    for postal in Counter(tmp.postal_code):
        postal_codes_list.remove(postal)

    # merge with location features
    tmp = pd.DataFrame(tmp.merge(postal_df, on='postal_code', how='left')).reset_index(drop=True)
    
    # add competitiveness and review (TO-DO)
    tmp['competitiveness']= ''
#     tmp['review']= ''
    for j in range(len(tmp)):
        
        # competitiveness
        place = all_df[all_df.postal_code == tmp['postal_code'][j]]
        tmp['competitiveness'][j] = -(len(Counter(place[place.categories == category].name))/len(Counter(place.name)))
        
        # TO-DO add review
    
    print(list(tmp.postal_code))
    # 看差距過大的店家
    if max(tmp.postal_code.astype(int))- min(tmp.postal_code.astype(int)) > 10000:
        print( max(tmp.postal_code.astype(int)) ,min(tmp.postal_code.astype(int)) ,\
              max(tmp.postal_code.astype(int))- min(tmp.postal_code.astype(int))) 
        # train test split
        tmp_train , tmp_test = train_test_split(tmp, test_size=0.33, shuffle= True,random_state=RANDOM_STATE)
        tmp_train = tmp_train.reset_index(drop=True)
        tmp_test = tmp_test.reset_index(drop=True)
        
        ## train 
        neg_df = pd.DataFrame()

        for j in range(len(tmp_train)):

            cnt =0 
            category = tmp_train.categories[j]

            # add the negative samples
            for k in range(len(postal_df)):
                if cnt<2:
                    if postal_df.postal_code[k] in postal_codes_list and\
                    abs(int(postal_df.postal_code[k])- int(tmp_train['postal_code'][j])) <= 1000:
                        cnt+=1
                        new = pd.DataFrame(tmp_train.iloc[j]).T.reset_index(drop=True)
                        postal_codes_list.remove(postal_df.postal_code[k])
                        new = create_new (new ,category,postal_df.postal_code[k], all_df, postal_df)
                        neg_df = pd.concat([neg_df ,new ])
                else:
                    break
            if int(tmp_train.postal_code[j])-min(neg_df.postal_code.astype(int))>1000:
                print(f'Something is wrong with the shape of the neg sample for train {i}')
                break
        print(f' over 100000 {i}pos : {tmp_train.postal_code}  train : {neg_df.postal_code}')

#         tmp_train = pd.concat([tmp_train ,neg_df ])

        ## test 

        neg_df = pd.DataFrame()
        for j in range(len(tmp_test)):
            cnt =0
            category = tmp_test.categories[j]

            # add negative samples
            for k in range(len(postal_df)):
                if cnt<2:
                    if postal_df.postal_code[k] in postal_codes_list and\
                    abs(int(postal_df.postal_code[k])-int(tmp_test['postal_code'][j])) <= 1000:
                        cnt+=1
                        new = pd.DataFrame(tmp_test.iloc[j]).T.reset_index(drop=True)
                        postal_codes_list.remove(postal_df.postal_code[k])
                        new = create_new (new ,category,postal_df.postal_code[k], all_df, postal_df)
                        neg_df = pd.concat([neg_df ,new ])
                else:
                    break
            if int(tmp_test.postal_code[j])-min(neg_df.postal_code.astype(int))>1000:
                print(f'Something is wrong with the shape of the neg sample for test {i}')
                break
        print(f' over 100000 {i} pos : {tmp_test.postal_code} test : {neg_df.postal_code}')
        print('============================================')
#         tmp_test = pd.concat([tmp_test ,neg_df ])
        
    else:
        print( max(tmp.postal_code.astype(int)) ,min(tmp.postal_code.astype(int)) ,\
              max(tmp.postal_code.astype(int))-min(tmp.postal_code.astype(int)) )
        # create negative 
        neg_df = pd.DataFrame()
        cnt = 0 

        for j in range(len(postal_df)):
            if len(neg_df) > 0 :
                print('this is neg',list(neg_df.postal_code))
            if cnt > 20 :
                break
            if postal_df.postal_code[j] in postal_codes_list and \
            abs(max(tmp.postal_code.astype(int)) - int(postal_df.postal_code[j])) <= 500:
                cnt+=1
                new = pd.DataFrame(tmp.iloc[0]).T.reset_index(drop=True)
                new = new.drop(columns=['density', 'entropy','area_pop', 'accessibility', 'complementary'])
                new['postal_code'] = postal_df.postal_code[j]
                postal_codes_list.remove(postal_df.postal_code[j])
                new = pd.DataFrame(new.merge(postal_df, on='postal_code', how='left'))
                new['relevance'] = 0
                # add competitiveness
                new['competitiveness']=''
                place = all_df[all_df.postal_code == postal_df.postal_code[j]].reset_index(drop=True)
                new['competitiveness'] = -(len(Counter(place[place.categories ==category].name))/len(Counter(place.name)))

                # TO-DO add review 
                neg_df = pd.concat([neg_df ,new ])
        
        print(f' below 10000 {i} pos : {tmp.postal_code}  : {neg_df.postal_code}')
        if max(tmp.postal_code.astype(int))-min(neg_df.postal_code.astype(int))>1000:
            print(f'Something is wrong with the shape of the neg sample for {i}')
            break
        print('============================================')
#     train_df = pd.concat([train_df,tmp_train])
#     test_df = pd.concat([test_df,tmp_test])
# print(f'Finished constructing....  train res cnt  : {len(Counter(train_df.name))} test res cnt : {len(Counter(test_df.name))}')

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


['46250', '46142', '46123', '19341', '19020', '08054']
46250 8054 38196
Something is wrong with the shape of the neg sample for train On The Border Mexican Grill & Cantina
 over 100000 On The Border Mexican Grill & Cantinapos : 0    46142
1    46250
2    19341
3    46123
Name: postal_code, dtype: object  train : 0    46032
0    46033
0    46037
0    46038
0    18901
0    18902
0    46040
0    46055
Name: postal_code, dtype: object
 over 100000 On The Border Mexican Grill & Cantina pos : 0    19020
1    08054
Name: postal_code, dtype: object test : 0    18073
0    18912
0    08002
0    08003
Name: postal_code, dtype: object
['89523', '70056', '34609', '33701']
89523 33701 55822
 over 100000 China Kitchenpos : 0    89523
1    34609
Name: postal_code, dtype: object  train : 0    89431
0    89432
0    33609
0    33610
Name: postal_code, dtype: object
Something is wrong with the shape of the neg sample for test China Kitchen
 over 100000 China Kitchen pos : 0    33701
1    70056
Name: posta

this is neg ['37012', '37013', '37015', '37024', '37027', '37043', '37064', '37066', '37067', '37069', '37072', '37073', '37075', '37076', '37080', '37082', '37086', '37090']
this is neg ['37012', '37013', '37015', '37024', '37027', '37043', '37064', '37066', '37067', '37069', '37072', '37073', '37075', '37076', '37080', '37082', '37086', '37090', '37115']
this is neg ['37012', '37013', '37015', '37024', '37027', '37043', '37064', '37066', '37067', '37069', '37072', '37073', '37075', '37076', '37080', '37082', '37086', '37090', '37115']
this is neg ['37012', '37013', '37015', '37024', '37027', '37043', '37064', '37066', '37067', '37069', '37072', '37073', '37075', '37076', '37080', '37082', '37086', '37090', '37115', '37135']
this is neg ['37012', '37013', '37015', '37024', '37027', '37043', '37064', '37066', '37067', '37069', '37072', '37073', '37075', '37076', '37080', '37082', '37086', '37090', '37115', '37135', '37138']
 below 10000 Burger Republic pos : 0    37211
1    37203
2    

this is neg ['18912', '18913', '18914', '18915', '18917', '18928', '18929', '18931', '18932', '18936', '18938', '18940', '18944', '18946', '18947', '18949']
this is neg ['18912', '18913', '18914', '18915', '18917', '18928', '18929', '18931', '18932', '18936', '18938', '18940', '18944', '18946', '18947', '18949', '18951']
this is neg ['18912', '18913', '18914', '18915', '18917', '18928', '18929', '18931', '18932', '18936', '18938', '18940', '18944', '18946', '18947', '18949', '18951', '18954']
this is neg ['18912', '18913', '18914', '18915', '18917', '18928', '18929', '18931', '18932', '18936', '18938', '18940', '18944', '18946', '18947', '18949', '18951', '18954', '18960']
this is neg ['18912', '18913', '18914', '18915', '18917', '18928', '18929', '18931', '18932', '18936', '18938', '18940', '18944', '18946', '18947', '18949', '18951', '18954', '18960', '18964']
this is neg ['18912', '18913', '18914', '18915', '18917', '18928', '18929', '18931', '18932', '18936', '18938', '18940', '189

this is neg ['33503', '33510', '33511']
this is neg ['33503', '33510', '33511', '33534']
this is neg ['33503', '33510', '33511', '33534', '33541']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542', '33544']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542', '33544', '33545']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542', '33544', '33545', '33547']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542', '33544', '33545', '33547', '33548']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542', '33544', '33545', '33547', '33548', '33549']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542', '33544', '33545', '33547', '33548', '33549', '33556']
this is neg ['33503', '33510', '33511', '33534', '33541', '33542', '33544', '33545', '33547', '33548', '33549', '33556', '33558'

...

 over 100000 Chuy'spos : 0    46032
1    46237
2    37203
3    37214
Name: postal_code, dtype: object  train : 0    46033
0    46037
0    46038
0    46040
0    37012
0    37013
0    37015
0    37024
Name: postal_code, dtype: object
 over 100000 Chuy's pos : 0    37067
1    33559
Name: postal_code, dtype: object test : 0    37027
0    37043
0    33503
0    33510
Name: postal_code, dtype: object
['19141', '19140', '19130']
19141 19130 11
this is neg ['18901']
this is neg ['18901', '18902']
this is neg ['18901', '18902', '18912']
this is neg ['18901', '18902', '18912', '18913']
this is neg ['18901', '18902', '18912', '18913', '18914']
this is neg ['18901', '18902', '18912', '18913', '18914', '18915']
this is neg ['18901', '18902', '18912', '18913', '18914', '18915', '18917']
this is neg ['18901', '18902', '18912', '18913', '18914', '18915', '18917', '18928']
this is neg ['18901', '18902', '18912', '18913', '18914', '18915', '18917', '18928', '18929']
this is neg ['18901', '18902', '18912'

this is neg ['70003', '70005', '70006', '70032', '70037', '70043', '70053', '70056', '70058', '70062', '70065', '70070', '70072', '70075', '70087', '70092']
this is neg ['70003', '70005', '70006', '70032', '70037', '70043', '70053', '70056', '70058', '70062', '70065', '70070', '70072', '70075', '70087', '70092', '70094']
this is neg ['70003', '70005', '70006', '70032', '70037', '70043', '70053', '70056', '70058', '70062', '70065', '70070', '70072', '70075', '70087', '70092', '70094', '70112']
this is neg ['70003', '70005', '70006', '70032', '70037', '70043', '70053', '70056', '70058', '70062', '70065', '70070', '70072', '70075', '70087', '70092', '70094', '70112', '70113']
this is neg ['70003', '70005', '70006', '70032', '70037', '70043', '70053', '70056', '70058', '70062', '70065', '70070', '70072', '70075', '70087', '70092', '70094', '70112', '70113', '70114']
this is neg ['70003', '70005', '70006', '70032', '70037', '70043', '70053', '70056', '70058', '70062', '70065', '70070', '700

this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706', '85707']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706', '85707', '85710']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706', '85707', '85710', '85711']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706', '85707', '85710', '85711', '85712']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706', '85707', '85710', '85711', '85712', '85713']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706', '85707', '85710', '85711', '85712', '85713', '85714']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85706', '85707', '85710', '85711', '85712', '85713', '85714', '8571

this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704', '85707']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704', '85707', '85710']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704', '85707', '85710']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704', '85707', '85710', '85712']
this is neg ['85614', '85619', '85629', '85641', '85653', '85658', '85701', '85702', '85704', '85707', '85710

this is neg ['34221']
this is neg ['34221', '34604']
this is neg ['34221', '34604']
this is neg ['34221', '34604', '34607']
this is neg ['34221', '34604', '34607', '34608']
this is neg ['34221', '34604', '34607', '34608', '34609']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610', '34619']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610', '34619', '34638']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610', '34619', '34638', '34639']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610', '34619', '34638', '34639', '34652']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610', '34619', '34638', '34639', '34652', '34653']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610', '34619', '34638', '34639', '34652', '34653', '34654']
this is neg ['34221', '34604', '34607', '34608', '34609', '34610', '34619', '34638', '34639', '34652', '34653',

In [ ]:
train_df.shape , test_df.shape

In [ ]:
for i in ['density', 'entropy', 'competitiveness','area_pop', 'accessibility','complementary','relevance']:
    train_df[i] = train_df[i].astype('float')
    test_df[i] = test_df[i].astype('float')
train_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_without_review_pointwise_v3_3.pkl')
test_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_without_review_pointwise_v3_3.pkl')